In [1]:
import pandas as pd
import eform_query_processor as eqp
import os

In [2]:
# setting up the absolute savepath
save_path = r'C:\Users\aange\OneDrive\Desktop\Personal Python Projects\ferc_data_project\ferc_714_Q4_2011'

save_path_new = r'C:\Users\AngelSalazar\python_scratch\ferc_data_project\ferc_714_xml_files_test'

In [3]:
# loading in df containing files we want to grab
source_df = pd.read_csv('process_check/eform_filtered_data_2024-11-09_2011.csv')

source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        145 non-null    object
 1   Company    145 non-null    object
 2   Form       145 non-null    object
 3   Year       145 non-null    int64 
 4   Period     145 non-null    object
 5   Date/Time  145 non-null    object
 6   Status     145 non-null    object
 7   Filing ID  145 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 9.2+ KB


In [4]:
# checking the first 5
source_df_f5 = source_df[:5]

print(source_df_f5.info())

print(source_df_f5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        5 non-null      object
 1   Company    5 non-null      object
 2   Form       5 non-null      object
 3   Year       5 non-null      int64 
 4   Period     5 non-null      object
 5   Date/Time  5 non-null      object
 6   Status     5 non-null      object
 7   Filing ID  5 non-null      int64 
dtypes: int64(2), object(6)
memory usage: 452.0+ bytes
None
       CID                                            Company      Form  Year  \
0  C003529                 Seminole Electric Cooperative, Inc  Form 714  2011   
1  C001184                      Tucson Electric Power Company  Form 714  2011   
2  C004245  Salt River Project Agricultural Improvement an...  Form 714  2011   
3  R714141                  City Utilities of Springfield, MO  Form 714  2011   
4  C011508  Metropolitan Water District of So

In [5]:
# create a list of filing id
filing_ids = list(source_df['Filing ID'])
print(filing_ids)

[15494, 15496, 15462, 15498, 15497, 15389, 15388, 15380, 15495, 15466, 15409, 15493, 15492, 15491, 15490, 15365, 15420, 15413, 15412, 15377, 15487, 15485, 15431, 15489, 15488, 15484, 15486, 15482, 15483, 15481, 15480, 15479, 15478, 15477, 15476, 15475, 15474, 15473, 15472, 15471, 15468, 15467, 15464, 15470, 15469, 15460, 15465, 15458, 15457, 15459, 15463, 15461, 15455, 15454, 15451, 15456, 15450, 15453, 15452, 15449, 15448, 15447, 15446, 15443, 15442, 15445, 15441, 15440, 15438, 15444, 15439, 15436, 15437, 15433, 15435, 15434, 15430, 15378, 15432, 15429, 15428, 15426, 15424, 15427, 15425, 15423, 15422, 15421, 15419, 15418, 15417, 15415, 15416, 15414, 15411, 15408, 15407, 15406, 15405, 15410, 15404, 15403, 15402, 15400, 15399, 15398, 15397, 15395, 15396, 15394, 15401, 15393, 15392, 15391, 15387, 15390, 15386, 15385, 15383, 15384, 15382, 15381, 15379, 15376, 15375, 15374, 15373, 15372, 15371, 15370, 15369, 15368, 15367, 15366, 15364, 15363, 15362, 15361, 15360, 15359, 15358, 15357, 15356

HERE WE ARE CREATING A FUNCTION THAT CAN TEST WHETHER OR NOT IT IS EASY TO PARALLELIZE FUNCTIONS

In [6]:
import multiprocessing
import concurrent.futures
import os


In [6]:
# return files
eqp.return_eform_filings(filing_ids=filing_ids, 
                         save_path=save_path)

Seminole Electric Cooperative, Inc_ferc_714_Q42011_15494.xml
Tucson Electric Power Company_ferc_714_Q42011_15496.xml
Salt River Project Agricultural Improvement and Power District_ferc_714_Q42011_15462.xml
City Utilities of Springfield, MO_ferc_714_Q42011_15498.xml
Metropolitan Water District of Southern California_ferc_714_Q42011_15497.xml
City of West Memphis_ferc_714_Q42011_15389.xml
City of Conway_ferc_714_Q42011_15388.xml
City of North Little Rock_ferc_714_Q42011_15380.xml
Puget Sound Energy, Inc._ferc_714_Q42011_15495.xml
Chugach Electric Association, Inc._ferc_714_Q42011_15466.xml
Progress Energy (Florida Power Corp.)_ferc_714_Q42011_15409.xml
Buckeye Power, Inc._ferc_714_Q42011_15493.xml
Florida Municipal Power Agency_ferc_714_Q42011_15492.xml
Golden Spread Electric Cooperative, Inc._ferc_714_Q42011_15491.xml
Tri-State Generation and Transmission Association, Inc._ferc_714_Q42011_15490.xml
Modesto Irrigation District_ferc_714_Q42011_15365.xml
Sacramento Municipal Utility Distri

Now we want to get a sense of how to sort our directories by time stamp
so that we can return a list of file paths to match with out pre existing
filing ids dataframe, allowing us to rename duplicate named files to assess
which files may have been revises or simply reuploaded at a later time.

In [7]:
# function to sort directory provided by either name or time
def sort_directory(sort_by: str, dir_path: str):

    '''
    In essence, first select whether you want to sort directory by name or by date created
    '''

    # purpose is to return the time the file was created
    def get_creation_time(item):
        item_path = os.path.join(dir_path, item)
        return os.path.getctime(item_path)
    
    # items meaning the files wihthin the directory path defined
    items = os.listdir(dir_path)

    if sort_by == 'date':
        '''
        in this case, user would like to sort directory path files by date
        '''
        print('Sorting files in path by creation date!')

        # sorting our items from oldest to newest
        sorted_items = sorted(items, key=get_creation_time)

        return sorted_items

    elif sort_by == 'name':
        '''
        in this case we would like to sort directory path files by name
        '''
        print('Sorting files in path by name!')

    else: 
        print('Please redefine how you would like to sort!')


In [8]:
# folder_path to test
fpath_714 = 'ferc_714_Q4_2020'

# attempt to check function
test_case = sort_directory(sort_by='date', dir_path=fpath_714)
print(test_case)


Sorting files in path by creation date!
['wk-20201231.xml', 'ArizonaPublicServiceCompany-116-2020Q4F714.xbrl', 'CaliforniaIndependentSystemOperator-125-2020Q4F714.xbrl', 'NaturenerWindWatchLlc-324-2020Q4F714.xbrl', 'GlacierWindBalancingAuthority-312-2020Q4F714.xbrl', 'BasinElectricPowerCooperative-287-2020Q4F714.xbrl', 'BuckeyePowerInc-124-2020Q4F714.xbrl', 'SeminoleElectricCooperativeInc-248-2020Q4F714.xbrl', 'ColoradoSpringsUtilities-143-2020Q4F714.xbrl', 'ProgressEnergyCarolinaPowerLightCompany-233-2020Q4F714.xbrl', 'BonnevillePowerAdministrationUsdoe-122-2020Q4F714.xbrl', 'SouthwesternPowerAdministrationDoe-258-2020Q4F714.xbrl', 'IsoNewEnglandInc-185-2020Q4F714.xbrl', 'ModestoIrrigationDistrict-206-2020Q4F714.xbrl', 'WesternAreaPowerAdministrationColoradoMissouriControlAreaRockyMtnRe-273-2020Q4F714.xbrl', 'LosAngelesDepartmentOfWaterAndPower-194-2020Q4F714.xbrl', 'MinnesotaMunicipalPowerAgency-283-2020Q4F714.xbrl', 'PudNo1OfDouglasCounty-238-2020Q4F714.xbrl', 'WesternAreaPowerAdmin

In [9]:
# function to change date times to the following format so that we can use it to rename files
# 12/9/2024, 4:25 PM --> 12_9_2024_4_25_PM 
# replace any [/], [,], [space] and [:] with an undescore

def clean_subnames(subname_list: list):
    '''
    given a string of a date or cname convert any defined chars to underscores
    '''
    # edited subname list
    new_subname_list = []

    for subname in subname_list:
        # first iteration checks for /
        iteration_1 = subname.replace("/", "_")
        iteration_2 = iteration_1.replace(",", "_")
        iteration_3 = iteration_2.replace(" ", "_")
        iteration_4 = iteration_3.replace(".", "_")
        final_iteration = iteration_4.replace(":", "_")
        new_subname_list.append(final_iteration)

    return new_subname_list


test_date_rename = clean_subnames(subname_list=["7/9/2024, 4:25 PM"])
cname_rename = clean_subnames(subname_list=['Southwest Power Pool, Inc.'])
print(test_date_rename)
print(cname_rename)

['7_9_2024__4_25_PM']
['Southwest_Power_Pool__Inc_']


In [10]:
# function aimed at setting up the renaming of files by grabbing info from df we used to grab files
def rename_xml_base(dframe: object):
    '''
    in essence, go through df and set up framework for the renaming of xml files grabbed
    '''
    # a finalized list of the renamed xml names
    renamed_xmls = []

    # columns to combine and combined strings list
    columns_to_combine = ['Form', 'Year', 'Company', 'Period', 'Date/Time', 'Filing ID']

    # combined strings
    for index, row in dframe.iterrows():

        # for each row in our dataframe we are combining select col values
        combined_string = ' '.join(row[columns_to_combine].astype(str))

        # in essence now changing all sub text into underscores
        final_string = clean_subnames(subname_list=list(combined_string))

        # combined final string where we have converted all sub elements in our words
        f_string = ''.join([str(item) for item in final_string])

        # printing combined string to see what's up
        renamed_xmls.append(f_string)
    
    return renamed_xmls



In [11]:
x = rename_xml_base(dframe=source_df)
print(x)


['Form_714_2020_Tennessee_Valley_Authority_Q4_8_21_2023__4_51_PM_133693', 'Form_714_2020_Arizona_Public_Service_Company_Q4_6_24_2021__10_50_AM_182136', 'Form_714_2020_California_Independent_System_Operator_Corporation_Q4_6_14_2021__11_16_AM_182163', 'Form_714_2020_BHE_Wind_Watch__LLC_Q4_6_3_2021__12_45_PM_182152', 'Form_714_2020_Glacier_Wind_Balancing_Authority_Q4_6_3_2021__12_34_PM_182153', 'Form_714_2020_Basin_Electric_Power_Cooperative_Q4_6_3_2021__9_45_AM_182162', 'Form_714_2020_Buckeye_Power__Inc__Q4_6_3_2021__8_04_AM_182160', 'Form_714_2020_Seminole_Electric_Cooperative__Inc_Q4_6_2_2021__1_11_PM_182155', 'Form_714_2020_Colorado_Springs_Utilities_Q4_6_2_2021__1_03_PM_182232', 'Form_714_2020_Progress_Energy_(Carolina_Power_&_Light_Company)_Q4_6_2_2021__11_30_AM_182121', 'Form_714_2020_Bonneville_Power_Administration_Q4_6_1_2021__5_45_PM_182159', 'Form_714_2020_Southwestern_Power_Administration_Q4_6_1_2021__5_04_PM_182158', 'Form_714_2020_ISO_New_England_Inc__Q4_6_1_2021__4_09_PM_18

okay, so now we have the tools to rename a file with a unique name that 
gives us information on who filed and exactly when, 

this makes it easy to parse through files that have been submitted multiple times
by specific entities....


now we want to work on 
(1) sorting the files within a directory
(2) iterating throuhg this file directory and renaming files with the processed
    file xml namees

we want to make sure we are accurately naming so we don't incorrecly name any files

In [12]:
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(f'files list: {files_list}')

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(f'temp_xmls: {temp_xmls}')  

    # testing to see if things are matched 
    for file in files_list:
        print(f'{file}, {temp_xmls[files_list.index(str(file))]}')


In [13]:
# PLACEHOLDER
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(files_list)

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(temp_xmls)  

    # now we want to iterate through each file and rename it
    for file in files_list: 
        # true file path
        true_file_path = f'{dir_path}/{file}'
        print(true_file_path)

        # new name
        print(f'new file name: {temp_xmls[file.index(str(file))]}')

        # rename file by grabbing location of the 'file' element
        new_fpath = os.rename(f'{true_file_path}', 
                              f'{dir_path}/{temp_xmls[files_list.index(str(file))]}.xml')




it looks like we need to split things between 
1. folder path
2. old file path
3. new file path

In [14]:
# folder_path to test
folder_path = 'ferc_714_Q4_2020'

rename_all_files(xml_df=source_df, dir_path=fpath_714)


here is what we are working with: 
inputted df:          CID                                            Company      Form  \
0    C004480                         Tennessee Valley Authority  Form 714   
1    C001436                     Arizona Public Service Company  Form 714   
2    C001183  California Independent System Operator Corpora...  Form 714   
3    C002869                                BHE Wind Watch, LLC  Form 714   
4    R714312                   Glacier Wind Balancing Authority  Form 714   
..       ...                                                ...       ...   
105  C003851                       Platte River Power Authority  Form 714   
106  C003808           Central Electric Power Cooperative, Inc.  Form 714   
107  C003554                     Indiana Municipal Power Agency  Form 714   
108  R714284                     Missouri River Energy Services  Form 714   
109  C001188           Wolverine Power Supply Cooperative, Inc.  Form 714   

     Year Period           